In [1]:
"""
Train optimization algos on a neural net
"""

'\nTrain optimization algos on a neural net\n'

In [2]:
import sys
import pdb
sys.path.append("C:\\users\\mccar\\miniconda3\\lib\\site-packages")
sys.path.append("c:\\users\\mccar\\appdata\\local\\programs\\python\\python311\\lib\\site-packages")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose as mlr
from sklearn.preprocessing import LabelEncoder

In [3]:
# Load data

# pd.set_option('display.max_columns', 60)
train_df = pd.read_csv("data/gtzan_music_genre/features_30_sec.csv")
train_df.head()
print(train_df.shape)
print(train_df.info())

(1000, 60)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   filename                 1000 non-null   object 
 1   length                   1000 non-null   int64  
 2   chroma_stft_mean         1000 non-null   float64
 3   chroma_stft_var          1000 non-null   float64
 4   rms_mean                 1000 non-null   float64
 5   rms_var                  1000 non-null   float64
 6   spectral_centroid_mean   1000 non-null   float64
 7   spectral_centroid_var    1000 non-null   float64
 8   spectral_bandwidth_mean  1000 non-null   float64
 9   spectral_bandwidth_var   1000 non-null   float64
 10  rolloff_mean             1000 non-null   float64
 11  rolloff_var              1000 non-null   float64
 12  zero_crossing_rate_mean  1000 non-null   float64
 13  zero_crossing_rate_var   1000 non-null   float64
 14  harmony_mean  

In [4]:
# Preprocessing
train_df = train_df.drop(['filename', 'length'], axis = 1)
y = train_df['label']
X = train_df.drop('label', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# More preprocessing
label_encoder = LabelEncoder()
scale = MinMaxScaler()
scaled_data = scale.fit_transform(x_train)
x_train_sc = pd.DataFrame(scaled_data, columns = x_train.columns).values
scaled_data = scale.fit_transform(x_test)
x_test_sc = pd.DataFrame(scaled_data, columns = x_test.columns).values
y_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

# One-hot encode target labels
num_classes = len(np.unique(y_train))  # Assuming there are 10 classes
 # One hot encode target values
y_train_one_hot = pd.get_dummies(y_train).values
y_test_one_hot = pd.get_dummies(y_test).values

In [40]:
# NN optimization SA
nn_model = mlr.NeuralNetwork(hidden_nodes=[32, 16, 8],
                                     activation='relu',
                                     algorithm='random_hill_climb',
                                     max_iters=50000,
                                     bias=True,
                                     is_classifier=True,
                                     learning_rate=0.5,
                                     early_stopping=False,
                                     max_attempts=100,
                                     random_state=42)

# Fit the model
nn_model.fit(x_train_sc, y_train_one_hot)

# Predict labels for train and test set
y_train_pred = nn_model.predict(x_train_sc)
y_test_pred = nn_model.predict(x_test_sc)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_one_hot, y_train_pred)
test_accuracy = accuracy_score(y_test_one_hot, y_test_pred)
print(train_accuracy)
print(test_accuracy)


0.69625
0.385


In [41]:
# Simulated Annealing
nn_model1 = mlr.NeuralNetwork(hidden_nodes = [32, 16, 8], activation = 'relu', 
                                 algorithm = 'simulated_annealing', 
                                 max_iters = 50000, bias = True, is_classifier = True, 
                                 learning_rate = 0.5, early_stopping = True, 
                                 clip_max = 5, max_attempts = 100, random_state = 42,
                                 schedule=mlr.ExpDecay())

# Fit the model
nn_model.fit(x_train_sc, y_train_one_hot)

# Predict labels for train and test set
y_train_pred = nn_model.predict(x_train_sc)
y_test_pred = nn_model.predict(x_test_sc)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_one_hot, y_train_pred)
test_accuracy = accuracy_score(y_test_one_hot, y_test_pred)
print(train_accuracy)
print(test_accuracy)


0.69625
0.385


In [42]:
nn_model1 = mlr.NeuralNetwork(hidden_nodes=[32, 16, 8], activation = 'relu', 
                                 algorithm = 'genetic_alg', 
                                 max_iters = 50000, bias = True, is_classifier = True, 
                                 learning_rate = 0.5, early_stopping = True, 
                                 clip_max = 5, max_attempts = 100, random_state = 3,
                                 pop_size=200, mutation_prob=0.1)


# Fit the model
nn_model.fit(x_train_sc, y_train_one_hot)

# Predict labels for train and test set
y_train_pred = nn_model.predict(x_train_sc)
y_test_pred = nn_model.predict(x_test_sc)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_one_hot, y_train_pred)
test_accuracy = accuracy_score(y_test_one_hot, y_test_pred)
print(train_accuracy)
print(test_accuracy)

0.69625
0.385


In [47]:
# NN optimization SA
nn_model = mlr.NeuralNetwork(hidden_nodes=[32, 16, 8],
                                     activation='relu',
                                     algorithm='gradient_descent',
                                     max_iters=10000,
                                     bias=True,
                                     is_classifier=True,
                                     learning_rate=0.2,
                                     early_stopping=False,
                                     max_attempts=100,
                                     random_state=42)

# Fit the model
nn_model.fit(x_train_sc, y_train_one_hot)

# Predict labels for train and test set
y_train_pred = nn_model.predict(x_train_sc)
y_test_pred = nn_model.predict(x_test_sc)

print(y_test_pred)
# Calculate accuracy
train_accuracy = accuracy_score(y_train_one_hot, y_train_pred)
test_accuracy = accuracy_score(y_test_one_hot, y_test_pred)
print(train_accuracy)
print(test_accuracy)


[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
0.1
0.1


In [ ]:
# Assuming x_train_sc has shape (number_of_samples, number_of_features)
# Make sure to replace 'number_of_features' with the actual number of features in your data.

# Define the model
model = Sequential()
# Add hidden layer
model.add(Dense(units=128, input_dim=x_train_sc.shape[1], activation='tanh'))
# Add hidden layer
model.add(Dense(units=64, activation='relu'))
# Add hidden layer
model.add(Dense(units=32, activation='relu'))
# Add hidden layer
model.add(Dense(units=16, activation='relu'))
# Add hidden layer
model.add(Dense(units=8, activation='relu'))
# Output layer
model.add(Dense(units=len(y_train.unique()), activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Print the model summary
# model.summary()
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
# model.summary()


# Custom callback for logging test accuracy at each epoch
class TestAccuracyLogger(Callback):
    def __init__(self, test_data):
        super().__init__()
        self.test_data = test_data
        self.test_accuracies = []

    def on_epoch_end(self, epoch, logs=None):
        _, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        self.test_accuracies.append(test_accuracy)
        print(f'Test Accuracy after Epoch {epoch + 1}: {test_accuracy * 100:.2f}%')

# Assuming x_test_sc and y_test are your test data and labels
# Also, replace 'num_classes' with the actual number of classes in your classification task.

# Create an instance of the custom callback
test_accuracy_logger = TestAccuracyLogger(test_data=(x_test_sc, y_test_encoded))

# Fit the model with the custom callback
history = model.fit(x_train_sc, y_encoded, epochs=250, batch_size=32, validation_split=0.1, callbacks=[test_accuracy_logger])

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(test_accuracy_logger.test_accuracies, label='Test Accuracy', linestyle='dashed')
plt.title('Learning Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# # Fit the model
# # print(x_train_sc)
# model.fit(x_train_sc, y_encoded, epochs=100, batch_size=32, validation_split=0.1)

# # # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(x_test_sc, y_test_encoded)
# print("Test Accuracy: %.2f%%" % (test_accuracy * 100))